<a href="https://colab.research.google.com/github/gnryu/ML22-2/blob/main/PHW1/PHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download the dataset using the wget command
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data

--2022-09-19 23:56:40--  https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19889 (19K) [application/x-httpd-php]
Saving to: ‘breast-cancer-wisconsin.data.3’

breast-cancer-wisco 100%[===================>]  19.42K   126KB/s    in 0.2s    

2022-09-19 23:56:41 (126 KB/s) - ‘breast-cancer-wisconsin.data.3’ saved [19889/19889]



In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold 

In [ ]:
data = pd.read_csv('breast-cancer-wisconsin.data', sep=',', header=None)
print(data)

          0   1   2   3   4   5   6   7   8   9   10
0    1000025   5   1   1   1   2   1   3   1   1   2
1    1002945   5   4   4   5   7  10   3   2   1   2
2    1015425   3   1   1   1   2   2   3   1   1   2
3    1016277   6   8   8   1   3   4   3   7   1   2
4    1017023   4   1   1   3   2   1   3   1   1   2
..       ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
694   776715   3   1   1   1   3   2   1   1   1   2
695   841769   2   1   1   1   2   1   1   1   1   2
696   888820   5  10  10   3   7   3   8  10   2   4
697   897471   4   8   6   4   3   4  10   6   1   4
698   897471   4   8   8   5   4   5  10   4   1   4

[699 rows x 11 columns]


In [ ]:
# drop the feature of 'ID'
data = data.drop([data.columns[0]], axis = 1)
print(data)

     1   2   3   4   5   6   7   8   9   10
0     5   1   1   1   2   1   3   1   1   2
1     5   4   4   5   7  10   3   2   1   2
2     3   1   1   1   2   2   3   1   1   2
3     6   8   8   1   3   4   3   7   1   2
4     4   1   1   3   2   1   3   1   1   2
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
694   3   1   1   1   3   2   1   1   1   2
695   2   1   1   1   2   1   1   1   1   2
696   5  10  10   3   7   3   8  10   2   4
697   4   8   6   4   3   4  10   6   1   4
698   4   8   8   5   4   5  10   4   1   4

[699 rows x 10 columns]


In [ ]:
# replace the value of ? to NaN
data.replace("?", np.NaN, inplace = True)
print(data.isna().sum())

1      0
2      0
3      0
4      0
5      0
6     16
7      0
8      0
9      0
10     0
dtype: int64


In [ ]:
# drop the row which has NaN value
data.dropna(inplace = True)

In [ ]:
data = data.astype(dtype = 'int64')
print(data.dtypes)

1     int64
2     int64
3     int64
4     int64
5     int64
6     int64
7     int64
8     int64
9     int64
10    int64
dtype: object


In [ ]:
X = data.iloc[:, :9]
print(X)

y = data.iloc[:,-1] # target feature
print(y)

     1   2   3  4  5   6   7   8  9
0    5   1   1  1  2   1   3   1  1
1    5   4   4  5  7  10   3   2  1
2    3   1   1  1  2   2   3   1  1
3    6   8   8  1  3   4   3   7  1
4    4   1   1  3  2   1   3   1  1
..  ..  ..  .. .. ..  ..  ..  .. ..
694  3   1   1  1  3   2   1   1  1
695  2   1   1  1  2   1   1   1  1
696  5  10  10  3  7   3   8  10  2
697  4   8   6  4  3   4  10   6  1
698  4   8   8  5  4   5  10   4  1

[683 rows x 9 columns]
0      2
1      2
2      2
3      2
4      2
      ..
694    2
695    2
696    4
697    4
698    4
Name: 10, Length: 683, dtype: int64


In [ ]:
# hyper parameter dictionary - for each model
hyper_parameter_dict = {
    'svm' : {'kernel' : ['linear', 'rbf'],
             'C' : [0.1, 1, 10],
             'gamma' : [0.001, 0.1, 1, 'scale', 'auto']
             },
    'DecisionTree': {
        "min_samples_leaf":[1, 2, 3, 4, 5, 6, 7, 8, 9],
        "max_depth":[2, 3, 4, 5, 6, None],
        "min_samples_split":[2, 3, 4, 5, 6, 7, 8, 9, 10]
    },
    'logistic':{
        'solver':['liblinear'], 
         'penalty':['l2', 'l1'],
        'C':[0.001, 0.01, 0.1, 1, 10]
    }
}

In [ ]:
from sklearn import svm

def SVM(X, Y, dt):

  model = svm.SVC()
      
  print("Params for grid search : ", dt)
  gs = GridSearchCV(model, param_grid=dt, cv=7, refit=True)
  gs.fit(X, Y)

  print("best_params: ", gs.best_params_)

  best_model = svm.SVC(kernel = gs.best_params_['kernel'], C = gs.best_params_['C'], gamma = gs.best_params_['gamma'])

  # Make best model with best parameters that the results of GridSearchCV
      
  return best_model

In [ ]:
from sklearn.linear_model import LogisticRegression

def Logistic(X, Y, dt):
  model = LogisticRegression()

  print("Params for grid search : ", dt)
  
  gs = GridSearchCV(model, param_grid=dt, cv=7, refit=True)
  gs.fit(X, Y)
  print("best_params: ", gs.best_params_)

  best_model = LogisticRegression(penalty = gs.best_params_['penalty'],solver = gs.best_params_['solver'], C = gs.best_params_['C'])
  

  return best_model


In [ ]:
from sklearn.tree import DecisionTreeClassifier

def DecisionTree(criterion, X, Y, dt):
  model = DecisionTreeClassifier(criterion=criterion)

  print("Params for grid search : ", dt)
  gs = GridSearchCV(model, param_grid=dt, cv=7, refit=True)
  gs.fit(X, Y)
  print("best_params: ", gs.best_params_)

  # Make best model with best parameters that the results of GridSearchCV
  best_model = DecisionTreeClassifier(criterion=criterion, max_depth=gs.best_params_['max_depth'], min_samples_split=gs.best_params_['min_samples_split'], min_samples_leaf=gs.best_params_['min_samples_leaf'])
  return best_model

In [ ]:
def main(X, target, model_code, scaling_TF, k, dict):
  # if 'scaling_TF' is true, call robust_scaling function
  print("n-fold(K) : ", k)
  print("Scaling : ", scaling_TF)
  if scaling_TF == True:
    scaler = RobustScaler()
    X = scaler.fit_transform(X)

  # 4 different model code
  if model_code == 0:
    # Decision tree (using entropy)
    print("Used model : Decision tree (using entropy)")
    model = DecisionTree('entropy', X, target, dict['DecisionTree'])
  elif model_code == 1:
    # Decision tree (using Gini index)
    print("Used model : Decision tree (using Gini index)")
    model = DecisionTree('gini', X, target, dict['DecisionTree'])
  elif model_code == 2:
    # Logistic regression
    print("Used model : Logistic regression")
    model = Logistic(X, target, dict['logistic'])
  elif model_code == 3:
    # Support vector machine
    print("Used model : Support vector machine")
    model = SVM(X, target, dict['svm'])

  # various numbers k for k-fold cross validation
  cv_scores = cross_val_score(model, X, target, cv = k)

  # print the average of cv_scores
  print('cv_scores mean : {}\n\n'.format(np.mean(cv_scores)))

In [ ]:
main(X, y, 0, False, 7, hyper_parameter_dict)
main(X, y, 0, True, 7, hyper_parameter_dict)

n-fold(K) :  7
Scaling :  False
Used model : Decision tree (using entropy)
Params for grid search :  {'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'max_depth': [2, 3, 4, 5, 6, None], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]}
best_params:  {'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 3}
cv_scores mean : 0.950347149168946


n-fold(K) :  7
Scaling :  True
Used model : Decision tree (using entropy)
Params for grid search :  {'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'max_depth': [2, 3, 4, 5, 6, None], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]}
best_params:  {'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 5}
cv_scores mean : 0.9518048751164679




In [ ]:
main(X, y, 1, False, 10, hyper_parameter_dict)
main(X, y, 1, True, 7, hyper_parameter_dict)

n-fold(K) :  10
Scaling :  False
Used model : Decision tree (using Gini index)
Params for grid search :  {'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'max_depth': [2, 3, 4, 5, 6, None], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]}
best_params:  {'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 4}
cv_scores mean : 0.941538789428815


n-fold(K) :  7
Scaling :  True
Used model : Decision tree (using Gini index)
Params for grid search :  {'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'max_depth': [2, 3, 4, 5, 6, None], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]}
best_params:  {'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 4}
cv_scores mean : 0.9532475729614378




In [ ]:
main(X, y, 2, False, 7, hyper_parameter_dict)
main(X, y, 2, True, 5, hyper_parameter_dict)

n-fold(K) :  7
Scaling :  False
Used model : Logistic regression
Params for grid search :  {'solver': ['liblinear'], 'penalty': ['l2', 'l1'], 'C': [0.001, 0.01, 0.1, 1, 10]}
best_params:  {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
cv_scores mean : 0.9649394367467163


n-fold(K) :  5
Scaling :  True
Used model : Logistic regression
Params for grid search :  {'solver': ['liblinear'], 'penalty': ['l2', 'l1'], 'C': [0.001, 0.01, 0.1, 1, 10]}
best_params:  {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}
cv_scores mean : 0.9722198368398454




In [ ]:
main(X, y, 3, False, 7, hyper_parameter_dict)
main(X, y, 3, True, 10, hyper_parameter_dict)

n-fold(K) :  7
Scaling :  False
Used model : Support vector machine
Params for grid search :  {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10], 'gamma': [0.001, 0.1, 1, 'scale', 'auto']}
best_params:  {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
cv_scores mean : 0.9693276426918338


n-fold(K) :  10
Scaling :  True
Used model : Support vector machine
Params for grid search :  {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10], 'gamma': [0.001, 0.1, 1, 'scale', 'auto']}
best_params:  {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
cv_scores mean : 0.9693094629156012


